In [9]:
import sys,os
%load_ext autoreload
%autoreload 2
print(sys.path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/usr/local/lib/python3.5/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.5/dist-packages/IPython/extensions', '/root/.ipython']


In [10]:
from pretrain import span_mask_utils
from pretrain import pretrain_helpers

In [13]:
from bunch import Bunch
import numpy as np
import tensorflow as tf


FLAGS = Bunch({})
FLAGS.min_tok = 3
FLAGS.max_tok = 10
FLAGS.sep_id = 102
FLAGS.pad_id = 0
FLAGS.cls_id = 101
FLAGS.mask_id = 103
FLAGS.batch_size = 1000
FLAGS.leak_ratio = 0.1
FLAGS.rand_ratio = 0.1
FLAGS.vocab_size = 21128
FLAGS.mask_prob = 0.15
FLAGS.sample_strategy = 'token_span'
FLAGS.confusion_matrix = None
FLAGS.confusion_mask_matrix = None
FLAGS.prepare_text_infilling = False
FLAGS.initial_ratio = 0.1
FLAGS.final_ratio = 0.3
FLAGS.num_train_steps = 1000

name_to_features = {
            "input_ori_ids":
                tf.FixedLenFeature([512], tf.int64)
}
record_spec = {
				"input_ori_ids":
						tf.FixedLenFeature([512], tf.int64),
# 				"input_mask":
# 						tf.FixedLenFeature([512], tf.int64),
				"segment_ids":
						tf.FixedLenFeature([512], tf.int64),
	}

def _decode_record(record, name_to_features, **kargs):
    example = tf.parse_single_example(record, name_to_features)
    return example

def train_input_fn(input_file, _parse_fn, name_to_features,
                   params,
                   num_predict=78, seq_len=512,
                   use_bfloat16=False,
                   truncate_seq=False,
                   stride=1,
                    **kargs):

    dataset = tf.data.TFRecordDataset(input_file, buffer_size=params.get("buffer_size", 100))
#     dataset = dataset.shuffle(1024)
#     dataset = dataset.map(lambda x:_parse_fn(x, record_spec))
    dataset = dataset.map(lambda x:span_mask_utils._decode_record(FLAGS, x, num_predict,
                                    seq_len, 
                                    use_bfloat16=use_bfloat16, 
                                    truncate_seq=truncate_seq, 
                                    stride=stride))
    dataset = dataset.batch(params.get("batch_size", 1))
    dataset = dataset.repeat(params.get("epoch", 1))
    iterator = dataset.make_one_shot_iterator()
    features = iterator.get_next()
    return features

output = ['/data/xuht/chinese_sub_task_0.tfrecord']
input_fn = train_input_fn(output[0], _decode_record, name_to_features, params=FLAGS)

sess = tf.Session()

init_op = tf.group(
            tf.local_variables_initializer())
sess.run(init_op)
cout = 0
while True:
    features = sess.run(input_fn)
    cout += 1
    break

INFO:tensorflow:Online sample with strategy: `token_span`.
==tf version== 1.12.0
==tf version== 1.12.0
INFO:tensorflow:masked_input: Tensor("Cast_23:0", shape=(512,), dtype=int32)
INFO:tensorflow:segment_ids: Tensor("Cast_24:0", shape=(512,), dtype=int32)
INFO:tensorflow:masked_lm_positions: Tensor("Cast_25:0", shape=(78,), dtype=int32)
INFO:tensorflow:masked_mask: Tensor("sub_6:0", shape=(512,), dtype=float32)
INFO:tensorflow:masked_lm_weights: Tensor("concat_1:0", shape=(78,), dtype=float32)
INFO:tensorflow:target_mapping: Tensor("Reshape_3:0", shape=(78, 512), dtype=float32)
INFO:tensorflow:target_mask: Tensor("concat_1:0", shape=(78,), dtype=float32)
INFO:tensorflow:target: Tensor("Cast_26:0", shape=(78,), dtype=int32)
INFO:tensorflow:masked_lm_ids: Tensor("Cast_27:0", shape=(78,), dtype=int32)
INFO:tensorflow:is_target: Tensor("Cast_28:0", shape=(512,), dtype=int32)
INFO:tensorflow:origin_input: Tensor("Cast_29:0", shape=(512,), dtype=int32)
INFO:tensorflow:pad_mask: Tensor("sub_7

In [14]:
unmasked_input_ids, _ = pretrain_helpers.scatter_update(
  tf.constant(features["masked_input"]), 
  tf.constant(features["masked_lm_ids"]), 
  tf.constant(features["masked_lm_positions"]))
resp = sess.run(unmasked_input_ids)

In [16]:
resp[0]-features['origin_input'][0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,